In [4]:
import requests
from xml.etree import ElementTree as ET

def pubmed_search(query, max_results=5):
    """
    Search PubMed for the given query and return a list of PubMed IDs.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "xml"
    }
    response = requests.get(base_url, params=params)
    root = ET.fromstring(response.content)
    id_list = [id_elem.text for id_elem in root.findall(".//Id")]
    return id_list

def pubmed_fetch_details(id_list):
    """
    Given a list of PubMed IDs, fetch details like title, authors, abstract.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    ids = ",".join(id_list)
    params = {
        "db": "pubmed",
        "id": ids,
        "retmode": "xml"
    }
    response = requests.get(base_url, params=params)
    root = ET.fromstring(response.content)
    
    papers = []
    for article in root.findall(".//PubmedArticle"):
        paper = {}
        article_meta = article.find("MedlineCitation/Article")
        if article_meta is not None:
            title_elem = article_meta.find("ArticleTitle")
            paper['title'] = title_elem.text if title_elem is not None else "No Title"
            
            abstract_elem = article_meta.find("Abstract/AbstractText")
            paper['abstract'] = abstract_elem.text if abstract_elem is not None else "No Abstract"
            
            authors = []
            for author in article_meta.findall("AuthorList/Author"):
                last = author.find("LastName")
                first = author.find("ForeName")
                if last is not None and first is not None:
                    authors.append(f"{first.text} {last.text}")
            paper['authors'] = authors
        papers.append(paper)
    return papers

# Example usage:
query = "diabetes and AI"
ids = pubmed_search(query)
print(f"Found {len(ids)} papers for query: '{query}'")

papers = pubmed_fetch_details(ids)

for i, paper in enumerate(papers, 1):
    print(f"\nPaper {i}:")
    print(f"Title: {paper['title']}")
    print(f"Authors: {', '.join(paper['authors']) if paper['authors'] else 'No authors found'}")
    print(f"Abstract: {paper['abstract'][:500]}...")  # print first 500 chars


Found 5 papers for query: 'diabetes and AI'

Paper 1:
Title: Kisspeptin Receptor Agonists and Antagonists: Strategies for Discovery and Implications for Human Health and Disease.
Authors: Xing Chen, Shu Yang, Natalie D Shaw, Menghang Xia
Abstract: The kisspeptin/kisspeptin receptor (...

Paper 2:
Title: Temporal Regulation of Early-Stage Cytokine Expression in Diabetic Wound Healing Under Negative Pressure Wound Therapy.
Authors: Hua-Sheng Chiu, Ting-Shuo Huang, Chien-Tzung Chen, Xin-Yu Lin, Po-Cheng Liao, Cai-Cin Liou, Chih-Chin Hsu, Sonal Somvanshi, Pavel Sumazin, Pang-Hung Hsu, Chi-Chin Sun, Yu-Chiau Shyu
Abstract: Negative pressure wound therapy (NPWT) is widely recognized for its efficacy in treating diabetic wounds, but the mechanisms involved in the wound healing process remain unclear. By examining changes in blood cytokine levels as molecular signaling precursors, we aim to provide a comprehensive cytokine profile to support adjunctive therapy research and clinical application